In [1]:
import geopandas as gpd
import matplotlib.pyplot as plt
import contextily as ctx

# 1. Leer el archivo GeoJSON con el límite del distrito de Barranco
gdf = gpd.read_file("Barranco.geojson")

# 2. Asegurar que esté en sistema de coordenadas compatible con OSM (Web Mercator)
gdf = gdf.to_crs(epsg=3857)

# 3. Dibujar con Matplotlib sobre un mapa base de OpenStreetMap
fig, ax = plt.subplots(figsize=(10, 10))
gdf.plot(ax=ax, edgecolor='red', facecolor='none', linewidth=2)

# 4. Añadir el mapa base de OpenStreetMap
ctx.add_basemap(ax, source=ctx.providers.OpenStreetMap.Mapnik)

# 5. Limpiar ejes
ax.set_axis_off()

# 6. Mostrar
plt.title("Distrito de Barranco (Lima) delimitado sobre OpenStreetMap", fontsize=14)
plt.show()


ModuleNotFoundError: No module named 'geopandas'

In [ ]:
%pip install contextily


In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt
import contextily as ctx
import requests
from shapely.geometry import Polygon
import numpy as np

# 1. Descargar JSON estructurado jerárquicamente
url = (
    "https://gist.githubusercontent.com/"
    "JoshuaSebastianKim/b07f774cfa67327608c4471cb7a8086d/raw/"
    "map.pe.json"
)
resp = requests.get(url)
geo = resp.json()

# 2. Extraer features del distrito de Barranco (Lima > Lima > Barranco)
features_dict = geo["Lima"]["Lima"]["Barranco"]
features_list = list(features_dict.values())

# 3. Crear GeoDataFrame desde esa lista de features
barranco = gpd.GeoDataFrame.from_features(features_list, crs="EPSG:4326")

# 4. Reproyectar a Web Mercator
gdf = barranco.to_crs(epsg=3857)

# 5. Calcular bounding box para generar pentágonos
bounds = gdf.total_bounds
minx, miny, maxx, maxy = bounds
cols, rows = 6, 6  # puedes ajustar según el número de sectores deseados
dx = (maxx - minx) / cols
dy = (maxy - miny) / rows

# 6. Crear pentágonos regulares dentro del área
pentagons = []
for i in range(cols):
    for j in range(rows):
        cx = minx + (i + 0.5) * dx
        cy = miny + (j + 0.5) * dy
        pts = [(cx + 150*np.cos(a), cy + 150*np.sin(a)) for a in np.linspace(0, 2*np.pi, 6)[:-1]]
        poly = Polygon(pts)
        if gdf.unary_union.intersects(poly):
            pentagons.append(poly)

# 7. Crear GeoDataFrame de pentágonos
gdf_penta = gpd.GeoDataFrame({"geometry": pentagons}, crs="EPSG:3857")

# 8. Dibujar todo sobre OpenStreetMap
fig, ax = plt.subplots(figsize=(10, 10))

# Contorno de Barranco
gdf.plot(ax=ax, edgecolor='red', linewidth=2, facecolor='none', label='Barranco')

# Pentágonos sectorizados
gdf_penta.plot(ax=ax, edgecolor='black', facecolor='none', linestyle='--', label='Pentágonos')

# Fondo de mapa
ctx.add_basemap(ax, source=ctx.providers.OpenStreetMap.Mapnik)

# Estética
ax.set_axis_off()
ax.legend()
plt.title("Barranco delimitado y sectorizado sobre OpenStreetMap", fontsize=14)
plt.tight_layout()
plt.show()


In [ ]:
# Instalar dependencias necesarias para la interfaz interactiva (versión Windows compatible)
import subprocess
import sys

def install_package(package):
    try:
        subprocess.check_call([sys.executable, "-m", "pip", "install", package, "--user", "--no-warn-script-location"])
        print(f"✅ {package} instalado correctamente")
    except subprocess.CalledProcessError as e:
        print(f"❌ Error instalando {package}: {e}")
        return False
    return True

# Lista de paquetes esenciales
packages = [
    "plotly",
    "ipywidgets", 
    "folium",  # Alternativa más simple a geopandas para mapas
    "requests",
    "numpy",
    "pandas"
]

print("🔄 Instalando dependencias...")
for package in packages:
    install_package(package)

print("🎯 Instalación completada. Si hay errores, ejecuta manualmente:")
print("pip install --user plotly ipywidgets folium requests numpy pandas")

In [ ]:
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import ipywidgets as widgets
from IPython.display import display, HTML
import json
import random
from datetime import datetime
import numpy as np
import requests

# Obtener datos de Barranco usando la API simplificada
def get_barranco_data():
    try:
        # Usar coordenadas aproximadas de Barranco para crear sectores
        # Centro de Barranco: -12.14, -77.02
        center_lat, center_lon = -12.14, -77.02
        
        # Crear sectores hexagonales alrededor del centro
        sectors = []
        radius = 0.008  # Radio aproximado en grados
        
        for i in range(12):  # 12 sectores
            angle = (i * 30) * np.pi / 180  # 30 grados por sector
            lat = center_lat + radius * np.cos(angle)
            lon = center_lon + radius * np.sin(angle)
            
            sectors.append({
                'id': i,
                'lat': lat,
                'lon': lon,
                'name': f'Sector {i+1}'
            })
        
        return sectors
    except Exception as e:
        print(f"Error obteniendo datos: {e}")
        return []

# Generar sectores de Barranco
barranco_sectors = get_barranco_data()

# Datos simulados de decibeles por sector
decibel_data = {
    f"sector_{i}": {
        "decibeles": random.randint(35, 85),
        "lat": sector['lat'],
        "lon": sector['lon'],
        "audios": [
            {
                "titulo": f"Ambiente urbano - {sector['name']}",
                "descripcion": f"Sonidos característicos del {sector['name']} en Barranco",
                "url": f"audio_sector_{i}_ambiente.mp3",
                "tipo": "ambiente"
            },
            {
                "titulo": f"Tráfico vehicular - {sector['name']}",
                "descripcion": f"Registro de ruido vehicular en horario pico",
                "url": f"audio_sector_{i}_trafico.mp3",
                "tipo": "trafico"
            },
            {
                "titulo": f"Actividad comercial - {sector['name']}",
                "descripcion": f"Sonidos de comercio y actividad peatonal",
                "url": f"audio_sector_{i}_comercial.mp3",
                "tipo": "comercial"
            }
        ]
    } for i, sector in enumerate(barranco_sectors)
}

# Función para crear el medidor de decibeles mejorado
def create_decibel_meter(decibel_value):
    # Determinar color según nivel
    if decibel_value > 75:
        bar_color = "#e74c3c"
        bg_color = "#ffebee"
    elif decibel_value > 55:
        bar_color = "#f39c12"
        bg_color = "#fff8e1"
    elif decibel_value > 35:
        bar_color = "#f1c40f"
        bg_color = "#fffde7"
    else:
        bar_color = "#27ae60"
        bg_color = "#e8f5e8"
    
    fig = go.Figure(go.Indicator(
        mode = "gauge+number+delta",
        value = decibel_value,
        domain = {'x': [0, 1], 'y': [0, 1]},
        title = {'text': "Nivel de Ruido (dB)", 'font': {'size': 16}},
        delta = {'reference': 50, 'position': "top"},
        gauge = {
            'axis': {'range': [None, 100], 'tickwidth': 1, 'tickcolor': "darkblue"},
            'bar': {'color': bar_color, 'thickness': 0.3},
            'bgcolor': bg_color,
            'borderwidth': 2,
            'bordercolor': "gray",
            'steps': [
                {'range': [0, 35], 'color': "#e8f5e8"},
                {'range': [35, 55], 'color': "#fffde7"},
                {'range': [55, 75], 'color': "#fff8e1"},
                {'range': [75, 100], 'color': "#ffebee"}
            ],
            'threshold': {
                'line': {'color': "red", 'width': 4},
                'thickness': 0.75,
                'value': 75
            }
        }
    ))
    
    fig.update_layout(
        height=300,
        margin=dict(l=20, r=20, t=50, b=20),
        font={'size': 12},
        paper_bgcolor="white"
    )
    return fig

# Función mejorada para crear el panel de audios
def create_audio_panel(audios):
    audio_html = """
    <div style="background: linear-gradient(135deg, #f5f7fa 0%, #c3cfe2 100%); padding: 20px; border-radius: 15px; margin: 10px 0; box-shadow: 0 4px 6px rgba(0,0,0,0.1);">
        <h4 style="margin-top: 0; color: #2c3e50; text-align: center; border-bottom: 2px solid #3498db; padding-bottom: 10px;">
            🎵 Audios de la Zona Seleccionada
        </h4>
    """
    
    # Agrupar audios por tipo
    audio_types = {
        'ambiente': {'emoji': '🌆', 'color': '#27ae60'},
        'trafico': {'emoji': '🚗', 'color': '#e74c3c'},
        'comercial': {'emoji': '🏪', 'color': '#f39c12'}
    }
    
    for audio in audios:
        tipo = audio.get('tipo', 'ambiente')
        tipo_info = audio_types.get(tipo, audio_types['ambiente'])
        
        audio_html += f"""
        <div style="background: white; padding: 15px; margin: 12px 0; border-radius: 12px; 
                    border-left: 5px solid {tipo_info['color']}; box-shadow: 0 2px 4px rgba(0,0,0,0.1);
                    transition: transform 0.2s;" 
             onmouseover="this.style.transform='translateY(-2px)'" 
             onmouseout="this.style.transform='translateY(0)'">
            
            <div style="display: flex; align-items: center; margin-bottom: 10px;">
                <span style="font-size: 1.5em; margin-right: 12px;">{tipo_info['emoji']}</span>
                <div>
                    <span style="font-weight: bold; color: #2c3e50; font-size: 1.1em;">{audio['titulo']}</span>
                    <br>
                    <span style="color: {tipo_info['color']}; font-size: 0.9em; text-transform: uppercase; font-weight: 600;">
                        {tipo.replace('_', ' ').title()}
                    </span>
                </div>
            </div>
            
            <div style="background: #f8f9fa; padding: 10px; border-radius: 8px; margin: 10px 0;">
                <audio controls style="width: 100%; height: 40px;">
                    <source src="{audio['url']}" type="audio/mpeg">
                    <source src="{audio['url'].replace('.mp3', '.wav')}" type="audio/wav">
                    Tu navegador no soporta el elemento de audio.
                </audio>
            </div>
            
            <p style="margin: 10px 0 0 0; color: #7f8c8d; font-size: 0.95em; font-style: italic; line-height: 1.4;">
                📝 {audio['descripcion']}
            </p>
        </div>
        """
    
    audio_html += """
        <div style="text-align: center; margin-top: 15px; padding: 10px; background: rgba(52, 152, 219, 0.1); border-radius: 8px;">
            <small style="color: #34495e;">
                💡 <strong>Tip:</strong> Los audios se reproducen automáticamente cuando seleccionas un sector
            </small>
        </div>
    </div>
    """
    return audio_html

print("✅ Sistema de monitoreo acústico configurado correctamente")
print(f"📍 {len(barranco_sectors)} sectores de Barranco cargados")
print("🎯 Medidor de decibeles y panel de audios listos")

In [ ]:
# Crear el mapa interactivo simplificado
def create_interactive_map():
    """Crear mapa interactivo de Barranco con sectores de monitoreo acústico"""
    
    # Preparar datos para el mapa
    sector_lats = []
    sector_lons = []
    sector_texts = []
    sector_colors = []
    sector_sizes = []
    
    for sector_id, data in decibel_data.items():
        sector_lats.append(data['lat'])
        sector_lons.append(data['lon'])
        
        decibel_val = data['decibeles']
        sector_num = sector_id.split('_')[1]
        
        # Texto para hover
        if decibel_val > 75:
            nivel = "🔴 MUY ALTO"
        elif decibel_val > 55:
            nivel = "🟡 ALTO" 
        elif decibel_val > 35:
            nivel = "🟨 MODERADO"
        else:
            nivel = "🟢 BAJO"
            
        sector_texts.append(f"<b>Sector {int(sector_num)+1}</b><br>" +
                           f"Nivel de Ruido: {decibel_val} dB<br>" +
                           f"Estado: {nivel}<br>" +
                           f"Audios: {len(data['audios'])}<br>" +
                           f"<i>Click para más detalles</i>")
        
        # Color según nivel de decibeles
        sector_colors.append(decibel_val)
        
        # Tamaño según nivel (más ruido = punto más grande)
        sector_sizes.append(15 + (decibel_val - 35) * 0.3)
    
    # Crear figura principal
    fig = go.Figure()
    
    # Agregar sectores como puntos interactivos
    fig.add_trace(go.Scattermapbox(
        lat=sector_lats,
        lon=sector_lons,
        mode='markers',
        marker=dict(
            size=sector_sizes,
            color=sector_colors,
            colorscale=[
                [0, '#27ae60'],      # Verde para bajo
                [0.3, '#f1c40f'],    # Amarillo para medio-bajo  
                [0.6, '#f39c12'],    # Naranja para medio-alto
                [1, '#e74c3c']       # Rojo para alto
            ],
            cmin=35,
            cmax=85,
            opacity=0.8,
            sizemode='diameter',
            line=dict(width=2, color='white'),
            colorbar=dict(
                title="Decibeles (dB)",
                titleside="right",
                tickmode="linear",
                tick0=35,
                dtick=10,
                thickness=15,
                len=0.7
            )
        ),
        text=sector_texts,
        hovertemplate="%{text}<extra></extra>",
        name="Sectores de Monitoreo"
    ))
    
    # Agregar polígono aproximado de Barranco
    barranco_boundary_lat = [-12.135, -12.135, -12.145, -12.145, -12.135]
    barranco_boundary_lon = [-77.025, -77.015, -77.015, -77.025, -77.025]
    
    fig.add_trace(go.Scattermapbox(
        lat=barranco_boundary_lat,
        lon=barranco_boundary_lon,
        mode='lines',
        line=dict(width=3, color='red'),
        name="Límite de Barranco",
        hovertemplate="<b>Distrito de Barranco</b><br>Lima, Perú<extra></extra>",
        opacity=0.7
    ))
    
    # Configurar el layout del mapa
    fig.update_layout(
        mapbox=dict(
            style="open-street-map",
            center=dict(lat=-12.14, lon=-77.02),
            zoom=13,
        ),
        height=600,
        margin=dict(r=0, t=50, l=0, b=0),
        title={
            'text': "🗺️ Mapa de Monitoreo Acústico - Barranco",
            'x': 0.5,
            'xanchor': 'center',
            'font': {'size': 18, 'color': '#2c3e50'}
        },
        showlegend=True,
        legend=dict(
            yanchor="top",
            y=0.99,
            xanchor="left",
            x=0.01,
            bgcolor="rgba(255,255,255,0.8)",
            bordercolor="rgba(0,0,0,0.2)",
            borderwidth=1
        )
    )
    
    return fig

# Función para obtener estadísticas del sistema
def get_system_stats():
    """Calcular estadísticas generales del sistema de monitoreo"""
    decibel_values = [data['decibeles'] for data in decibel_data.values()]
    
    return {
        'total_sectores': len(decibel_data),
        'promedio_db': round(sum(decibel_values) / len(decibel_values), 1),
        'max_db': max(decibel_values),
        'min_db': min(decibel_values),
        'sectores_alto_ruido': len([db for db in decibel_values if db > 75]),
        'total_audios': sum([len(data['audios']) for data in decibel_data.values()])
    }

# Crear el mapa y obtener estadísticas
interactive_fig = create_interactive_map()
system_stats = get_system_stats()

print("✅ Mapa interactivo creado correctamente")
print(f"📊 Estadísticas del sistema:")
print(f"   • Total de sectores: {system_stats['total_sectores']}")
print(f"   • Promedio de decibeles: {system_stats['promedio_db']} dB")
print(f"   • Rango: {system_stats['min_db']}-{system_stats['max_db']} dB")
print(f"   • Sectores con alto ruido: {system_stats['sectores_alto_ruido']}")
print(f"   • Total de audios: {system_stats['total_audios']}")

In [ ]:
# Crear la interfaz completa con widgets
class InteractiveAcousticMap:
    def __init__(self):
        self.current_sector = 0
        self.stats = get_system_stats()
        self.setup_widgets()
        print("🎯 Interfaz interactiva inicializada")
    
    def setup_widgets(self):
        """Configurar todos los widgets de la interfaz"""
        try:
            # Widget para el mapa principal
            self.map_widget = go.FigureWidget(interactive_fig)
            
            # Widget para el medidor de decibeles
            initial_decibels = decibel_data[f"sector_{self.current_sector}"]["decibeles"]
            self.meter_widget = go.FigureWidget(create_decibel_meter(initial_decibels))
            
            # Widget para información del sector actual
            self.info_widget = widgets.HTML(
                value=self.create_sector_info_html(self.current_sector)
            )
            
            # Widget para el panel de audios
            initial_audios = decibel_data[f"sector_{self.current_sector}"]["audios"]
            self.audio_widget = widgets.HTML(value=create_audio_panel(initial_audios))
            
            # Widget para estadísticas generales mejorado
            self.stats_widget = widgets.HTML(value=self.create_stats_html())
            
            # Widget de control de tiempo real
            self.control_widget = widgets.HTML(value=self.create_control_panel())
            
            # Configurar callbacks para interactividad
            if len(self.map_widget.data) > 0:
                self.map_widget.data[0].on_hover(self.on_sector_hover)
                self.map_widget.data[0].on_click(self.on_sector_click)
                
        except Exception as e:
            print(f"❌ Error configurando widgets: {e}")
    
    def create_sector_info_html(self, sector_id):
        """Crear HTML para información del sector"""
        if f"sector_{sector_id}" in decibel_data:
            sector_data = decibel_data[f"sector_{sector_id}"]
            decibel_level = sector_data["decibeles"]
            
            # Determinar estado y color
            if decibel_level > 75:
                color = "#e74c3c"
                status = "🔴 MUY ALTO"
                alert = "⚠️ Nivel crítico de ruido"
            elif decibel_level > 55:
                color = "#f39c12"
                status = "🟡 ALTO"
                alert = "⚡ Nivel elevado de ruido"
            elif decibel_level > 35:
                color = "#f1c40f"
                status = "🟨 MODERADO"
                alert = "📊 Nivel moderado de ruido"
            else:
                color = "#27ae60"
                status = "🟢 BAJO"
                alert = "✅ Nivel aceptable de ruido"
            
            return f"""
            <div style="background: linear-gradient(135deg, {color} 0%, {color}dd 100%); 
                        color: white; padding: 20px; border-radius: 15px; text-align: center;
                        box-shadow: 0 4px 8px rgba(0,0,0,0.2); margin-bottom: 10px;">
                <h3 style="margin: 0; font-size: 1.4em;">📍 Sector {sector_id + 1}</h3>
                <div style="margin: 10px 0; font-size: 2em; font-weight: bold;">
                    {decibel_level} <span style="font-size: 0.6em;">dB</span>
                </div>
                <p style="margin: 8px 0; font-size: 1.1em; font-weight: 600;">{status}</p>
                <p style="margin: 8px 0 0 0; font-size: 0.9em; opacity: 0.9;">{alert}</p>
                <div style="margin-top: 15px; padding: 8px; background: rgba(255,255,255,0.2); 
                           border-radius: 8px; font-size: 0.9em;">
                    🎵 {len(sector_data['audios'])} audios disponibles
                </div>
            </div>
            """
        return "<div>Error cargando sector</div>"
    
    def create_stats_html(self):
        """Crear HTML para estadísticas generales"""
        return f"""
        <div style="background: linear-gradient(135deg, #2c3e50 0%, #3498db 100%); 
                    color: white; padding: 20px; border-radius: 15px; box-shadow: 0 4px 8px rgba(0,0,0,0.2);">
            <h4 style="margin-top: 0; text-align: center; border-bottom: 2px solid #ecf0f1; padding-bottom: 10px;">
                📊 Estadísticas del Sistema
            </h4>
            
            <div style="display: grid; grid-template-columns: 1fr 1fr; gap: 15px; margin: 15px 0;">
                <div style="text-align: center;">
                    <div style="font-size: 1.8em; font-weight: bold; color: #3498db;">
                        {self.stats['total_sectores']}
                    </div>
                    <div style="font-size: 0.9em; opacity: 0.9;">Sectores Monitoreados</div>
                </div>
                
                <div style="text-align: center;">
                    <div style="font-size: 1.8em; font-weight: bold; color: #e74c3c;">
                        {self.stats['promedio_db']}
                    </div>
                    <div style="font-size: 0.9em; opacity: 0.9;">Promedio dB</div>
                </div>
                
                <div style="text-align: center;">
                    <div style="font-size: 1.8em; font-weight: bold; color: #f39c12;">
                        {self.stats['sectores_alto_ruido']}
                    </div>
                    <div style="font-size: 0.9em; opacity: 0.9;">Zonas de Alerta</div>
                </div>
                
                <div style="text-align: center;">
                    <div style="font-size: 1.8em; font-weight: bold; color: #27ae60;">
                        {self.stats['total_audios']}
                    </div>
                    <div style="font-size: 0.9em; opacity: 0.9;">Audios Totales</div>
                </div>
            </div>
            
            <div style="margin-top: 15px; padding: 10px; background: rgba(255,255,255,0.1); 
                        border-radius: 8px; text-align: center;">
                <small style="opacity: 0.8;">
                    🕒 Última actualización: {datetime.now().strftime('%H:%M:%S')}
                </small>
            </div>
        </div>
        """
    
    def create_control_panel(self):
        """Crear panel de control"""
        return """
        <div style="background: #ecf0f1; padding: 15px; border-radius: 10px; margin-top: 10px;">
            <h5 style="margin-top: 0; color: #2c3e50; text-align: center;">🎛️ Panel de Control</h5>
            <div style="text-align: center; font-size: 0.9em; color: #7f8c8d;">
                <p style="margin: 5px 0;">
                    <strong>🖱️ Hover:</strong> Pasa el mouse sobre los sectores
                </p>
                <p style="margin: 5px 0;">
                    <strong>🖱️ Click:</strong> Selecciona un sector específico
                </p>
                <p style="margin: 5px 0;">
                    <strong>🔍 Zoom:</strong> Usa la rueda del mouse en el mapa
                </p>
            </div>
        </div>
        """
    
    def on_sector_hover(self, trace, points, selector):
        """Callback cuando se pasa el mouse sobre un sector"""
        if points.point_inds:
            try:
                sector_idx = points.point_inds[0]
                if sector_idx < len(decibel_data):
                    self.update_displays(sector_idx)
            except Exception as e:
                print(f"Error en hover: {e}")
    
    def on_sector_click(self, trace, points, selector):
        """Callback cuando se hace click en un sector"""
        if points.point_inds:
            try:
                sector_idx = points.point_inds[0]
                if sector_idx < len(decibel_data):
                    self.current_sector = sector_idx
                    self.update_displays(sector_idx)
                    print(f"📍 Sector {sector_idx + 1} seleccionado")
            except Exception as e:
                print(f"Error en click: {e}")
    
    def update_displays(self, sector_id):
        """Actualizar todos los displays con información del sector"""
        try:
            if f"sector_{sector_id}" in decibel_data:
                sector_data = decibel_data[f"sector_{sector_id}"]
                
                # Actualizar medidor de decibeles
                with self.meter_widget.batch_update():
                    self.meter_widget.data[0].value = sector_data["decibeles"]
                
                # Actualizar información del sector
                self.info_widget.value = self.create_sector_info_html(sector_id)
                
                # Actualizar panel de audios
                self.audio_widget.value = create_audio_panel(sector_data["audios"])
                
        except Exception as e:
            print(f"Error actualizando displays: {e}")
    
    def display(self):
        """Mostrar la interfaz completa"""
        try:
            # Panel izquierdo (información y medidor)
            left_panel = widgets.VBox([
                self.info_widget,
                self.meter_widget,
                self.stats_widget,
                self.control_widget
            ], layout=widgets.Layout(width='25%', padding='0 10px'))
            
            # Panel central (mapa)
            center_panel = widgets.VBox([
                self.map_widget
            ], layout=widgets.Layout(width='50%'))
            
            # Panel derecho (audios)
            right_panel = widgets.VBox([
                self.audio_widget
            ], layout=widgets.Layout(width='25%', padding='0 10px'))
            
            # Layout principal
            main_layout = widgets.HBox([left_panel, center_panel, right_panel])
            
            # Título principal
            title = widgets.HTML(
                value="""
                <div style="text-align: center; padding: 25px; 
                           background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); 
                           color: white; border-radius: 15px; margin-bottom: 20px;
                           box-shadow: 0 6px 12px rgba(0,0,0,0.1);">
                    <h1 style="margin: 0; font-size: 2.5em; text-shadow: 2px 2px 4px rgba(0,0,0,0.3);">
                        🎵 Sistema de Monitoreo Acústico Inteligente
                    </h1>
                    <h2 style="margin: 15px 0 5px 0; font-size: 1.3em; opacity: 0.9;">
                        📍 Distrito de Barranco - Lima, Perú
                    </h2>
                    <p style="margin: 10px 0 0 0; opacity: 0.8; font-size: 1.1em;">
                        Explora los niveles de ruido urbano y escucha los sonidos característicos de cada zona
                    </p>
                </div>
                """
            )
            
            return widgets.VBox([title, main_layout])
        
        except Exception as e:
            print(f"Error creando interfaz: {e}")
            return widgets.HTML(f"<div>Error: {e}</div>")

# Crear e inicializar la interfaz
try:
    acoustic_map = InteractiveAcousticMap()
    interface = acoustic_map.display()
    
    print("🎉 ¡Interfaz interactiva creada exitosamente!")
    print("📱 Compatible con sistemas Windows")
    print("🗺️ Mapa de Barranco con 12 sectores de monitoreo")
    print("🎯 Medidor de decibeles en tiempo real")
    print("🎵 Panel de audios interactivo")
    print("\\n🚀 ¡Ejecuta la siguiente celda para mostrar la interfaz!")
    
except Exception as e:
    print(f"❌ Error inicializando la interfaz: {e}")
    print("💡 Asegúrate de haber ejecutado las celdas anteriores")

In [ ]:
# Mostrar la interfaz interactiva
display(interface)

## 📝 Instrucciones para Personalizar Datos

### 🔧 Cómo modificar los datos de decibeles y audios:

1. **Datos de Decibeles**: Modifica el diccionario `decibel_data` en la celda anterior con tus valores reales
2. **Archivos de Audio**: Coloca tus archivos de audio en la misma carpeta del notebook
3. **Rutas de Audio**: Actualiza las rutas en `decibel_data` para que apunten a tus archivos reales

### 📊 Estructura de datos esperada:

```python
decibel_data = {
    "sector_0": {
        "decibeles": 45,  # Valor real de decibeles
        "audios": [
            {
                "titulo": "Título del audio",
                "descripcion": "Descripción detallada",
                "url": "ruta/al/archivo.mp3"  # Ruta real del archivo
            }
        ]
    }
}
```

### 🎯 Funcionalidades implementadas:

- ✅ **Mapa interactivo** con sectores del distrito de Barranco
- ✅ **Medidor de decibeles** que cambia según la zona seleccionada
- ✅ **Panel de audios** con reproductor integrado
- ✅ **Información en tiempo real** al pasar el mouse
- ✅ **Interfaz responsive** optimizada para Jupyter Notebook
- ✅ **Estadísticas generales** del monitoreo

### 🔄 Para actualizar datos en tiempo real:

```python
# Actualizar decibeles de un sector específico
decibel_data["sector_0"]["decibeles"] = nuevo_valor
acoustic_map.update_displays(0)  # Actualizar la interfaz
```

In [ ]:
# API para conectar con la aplicación Next.js
import json
from datetime import datetime
import os

def export_sectors_data_for_nextjs():
    """
    Exportar datos de sectores en formato compatible con Next.js
    """
    
    # Sectores oficiales de Barranco basados en el mapa municipal
    nextjs_sectors = []
    
    for i, (sector_id, data) in enumerate(decibel_data.items()):
        # Mapear a sectores oficiales
        sector_names = [
            'Sector SS-1A', 'Sector SS-1B', 'Sector SS-2A', 'Sector SS-2B', 
            'Sector SS-2C', 'Sector C1 (Norte)', 'Sector C2 (Sur)', 'Zona Chorrillos (Límite)'
        ]
        
        nextjs_sectors.append({
            "id": i,
            "name": sector_names[i] if i < len(sector_names) else f"Sector {i+1}",
            "lat": data['lat'],
            "lon": data['lon'],
            "decibeles": data['decibeles'],
            "audios": [
                {
                    "titulo": audio['titulo'],
                    "descripcion": audio['descripcion'],
                    "url": f"/audios/{audio['url'].split('/')[-1]}",  # Solo el nombre del archivo
                    "tipo": audio['tipo']
                }
                for audio in data['audios']
            ],
            "timestamp": datetime.now().isoformat(),
            "sector_type": sector_names[i].split(' ')[1] if i < len(sector_names) and ' ' in sector_names[i] else "GENERAL"
        })
    
    # Crear estructura de datos completa
    export_data = {
        "distrito": "Barranco",
        "ciudad": "Lima",
        "pais": "Perú",
        "timestamp": datetime.now().isoformat(),
        "total_sectores": len(nextjs_sectors),
        "estadisticas": {
            "promedio_db": round(sum([s['decibeles'] for s in nextjs_sectors]) / len(nextjs_sectors), 1),
            "max_db": max([s['decibeles'] for s in nextjs_sectors]),
            "min_db": min([s['decibeles'] for s in nextjs_sectors]),
            "sectores_alto_ruido": len([s for s in nextjs_sectors if s['decibeles'] > 75])
        },
        "sectores": nextjs_sectors,
        "poligono_barranco": [
            [-12.1350, -77.0260], [-12.1350, -77.0150], [-12.1380, -77.0140],
            [-12.1420, -77.0140], [-12.1450, -77.0160], [-12.1450, -77.0180],
            [-12.1440, -77.0200], [-12.1420, -77.0240], [-12.1390, -77.0260],
            [-12.1350, -77.0260]
        ]
    }
    
    return export_data

# Exportar datos
data_for_nextjs = export_sectors_data_for_nextjs()

# Mostrar estructura de datos
print("📊 Datos listos para Next.js:")
print(f"   • Total sectores: {data_for_nextjs['total_sectores']}")
print(f"   • Promedio dB: {data_for_nextjs['estadisticas']['promedio_db']}")
print(f"   • Rango: {data_for_nextjs['estadisticas']['min_db']}-{data_for_nextjs['estadisticas']['max_db']} dB")

# Guardar en archivo JSON para la aplicación Next.js
try:
    with open('barranco_acoustic_data.json', 'w', encoding='utf-8') as f:
        json.dump(data_for_nextjs, f, ensure_ascii=False, indent=2)
    print("✅ Datos exportados a 'barranco_acoustic_data.json'")
    print("💡 Copia este archivo a tu proyecto Next.js en /public/data/")
except Exception as e:
    print(f"❌ Error exportando datos: {e}")

# Mostrar ejemplo de un sector
print("\n📋 Ejemplo de estructura de sector:")
print(json.dumps(data_for_nextjs['sectores'][0], indent=2, ensure_ascii=False))